# Reduccion de Suma Precipitacion imagen google Earth Engine

In [1]:
# Importar GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar shp cuenca
cuenca = ee.FeatureCollection("users/bravomoralesnino/SHP/Cuenca")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [4]:
geometria = cuenca.geometry()
Map.centerObject(geometria, 8)

In [5]:
Map.addLayer(cuenca, {"color" : "blue"},name = "Cuenca")
Map

Map(center=[-13.822942378556581, -71.5432582449152], controls=(WidgetControl(options=['position', 'transparent…

## Collección datos precipitación

In [6]:
# cargar datos de precipitacion (mm/Anual): 365 imagenes por anual
precipCollection = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
                        .select('precipitation')\
                        .filterDate('2019-01-01', '2019-12-31')

In [7]:
count = precipCollection.size()
count.getInfo()

364

In [8]:
# funcion para determinar la fecha de las imagenes
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [9]:
fecha = list_coll(precipCollection).getInfo()
print(fecha)

['2019/01/01', '2019/01/02', '2019/01/03', '2019/01/04', '2019/01/05', '2019/01/06', '2019/01/07', '2019/01/08', '2019/01/09', '2019/01/10', '2019/01/11', '2019/01/12', '2019/01/13', '2019/01/14', '2019/01/15', '2019/01/16', '2019/01/17', '2019/01/18', '2019/01/19', '2019/01/20', '2019/01/21', '2019/01/22', '2019/01/23', '2019/01/24', '2019/01/25', '2019/01/26', '2019/01/27', '2019/01/28', '2019/01/29', '2019/01/30', '2019/01/31', '2019/02/01', '2019/02/02', '2019/02/03', '2019/02/04', '2019/02/05', '2019/02/06', '2019/02/07', '2019/02/08', '2019/02/09', '2019/02/10', '2019/02/11', '2019/02/12', '2019/02/13', '2019/02/14', '2019/02/15', '2019/02/16', '2019/02/17', '2019/02/18', '2019/02/19', '2019/02/20', '2019/02/21', '2019/02/22', '2019/02/23', '2019/02/24', '2019/02/25', '2019/02/26', '2019/02/27', '2019/02/28', '2019/03/01', '2019/03/02', '2019/03/03', '2019/03/04', '2019/03/05', '2019/03/06', '2019/03/07', '2019/03/08', '2019/03/09', '2019/03/10', '2019/03/11', '2019/03/12', '2019

In [10]:
# Simbologia Precipitacion diario
viz_PP = {
    'min': 1,
    'max': 17,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

In [11]:
# Visualizar 
Map.addLayer(precipCollection, viz_PP, name = "Precippitacion")
Map

Map(bottom=35609.0, center=[-13.822942378556581, -71.5432582449152], controls=(WidgetControl(options=['positio…

## Suma de precipitaciones diarias

In [12]:
# reducir la image collection a una sola imagen sumando los 364 patrones diarios
annualPrecip = precipCollection.reduce(ee.Reducer.sum())

In [13]:
# Simbologia precipitacion acumulada anual
viz_PP_anual = {
    'min': 60,
    'max': 3000,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

In [14]:
# Visualizar 
Map.addLayer(annualPrecip, viz_PP_anual, name = "annualPrecip")
Map

Map(bottom=2539.0, center=[-16.55196172197251, -60.95214843750001], controls=(WidgetControl(options=['position…

## Recortar segun zona

In [15]:
# Recortar segun zona de estudio
annualPrecip_clip = annualPrecip.clip(cuenca)

In [16]:
# Visualizar 
Map.addLayer(annualPrecip_clip, viz_PP_anual, name = "annualPrecip_clip")
Map

Map(bottom=1435.0, center=[-19.145168196205297, -45.79101562500001], controls=(WidgetControl(options=['positio…

## Exportar Imagen calibrado

In [17]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [18]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [19]:
# Exportar imagen Precipitacion acumulada
geemap.ee_export_image(annualPrecip_clip, filename = "annualPrecip_clip.tif", scale=5000, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\annualPrecip_clip.tif


In [20]:
# Exportar en google drive
geemap.ee_export_image_to_drive(annualPrecip_clip, description='annualPrecip_clip', folder='GEE_Python', region=geometria, scale=5000)

Exporting annualPrecip_clip ...
